In [1]:
%load_ext autoreload
%autoreload 2

- Test data model

In [2]:
import sys

sys.path.insert(0, "..")

import pandas as pd
from src.domain.model_request import ModelRequest, SequenceModelRequest

In [3]:
mock_simple_df = pd.DataFrame({
    'user_id': [1, 2, 3]
})  

simple_input_data = ModelRequest.from_df_for_rec(mock_simple_df, user_col='user_id')

print(simple_input_data)

2025-06-27 02:22:49.500 | INFO     | src.domain.model_request:from_df_for_rec:36 - Use user_col=user_id


ModelRequest(user_id=tensor([1, 2, 3]), target_item=None, recommendation=True)


In [4]:
mock_sequence_df = pd.DataFrame({
    'user_id': [1, 2, 3],
    'item_sequence': [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
})

sequence_input_data = SequenceModelRequest.from_df_for_rec(mock_sequence_df, user_col='user_id', item_sequence_col='item_sequence')
print(sequence_input_data)

2025-06-27 02:22:49.517 | INFO     | src.domain.model_request:from_df_for_rec:51 - Use user_col=user_id
2025-06-27 02:22:49.517 | INFO     | src.domain.model_request:from_df_for_rec:52 - Use item_sequence_col=item_sequence


SequenceModelRequest(user_id=tensor([1, 2, 3]), target_item=None, recommendation=True, item_sequence=tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]))


- Test recommendation metrics

In [5]:
from src.algo.sequence.model import SequenceRatingPrediction
from src.eval.recommendation import RankingMetricComputer

In [6]:
mock_sequence_df = pd.DataFrame({
    'user_indice': [0, 1, 2],
    'item_sequence': [[0, 1, 2], [0, 1, 2], [0, 1, 2]],\
    "rating": [5, 4, 3],
    "timestamp": [1000, 2000, 3000],
    "item_indice": [0, 1, 2]
})

num_users = mock_sequence_df["user_indice"].nunique()
num_items = mock_sequence_df["item_sequence"].explode().nunique()
embedding_dim = 8
mock_model = SequenceRatingPrediction(num_users=num_users, num_items=num_items, embedding_dim=embedding_dim)

rec_metric_computer = RankingMetricComputer(
    rec_model= mock_model,
    top_k= 2,
    top_K=3
)

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:385: UserWarning:

enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.activation_relu_or_gelu was not True

2025-06-27 02:22:50.840 | INFO     | src.algo.sequence.model:__init__:122 - Start token used: 2, Padding token used: 3


In [7]:
rec_metric_computer.calculate(mock_sequence_df)

2025-06-27 02:22:50.863 | INFO     | src.domain.model_request:from_df_for_rec:51 - Use user_col=user_indice


2025-06-27 02:22:50.864 | INFO     | src.domain.model_request:from_df_for_rec:52 - Use item_sequence_col=item_sequence


Generating recommendations:   0%|          | 0/12 [00:00<?, ?it/s]

{'metrics': [{'metric': 'NDCGKMetric',
   'result': {'k': 2,
    'current': 1    0.333333
    2    0.333333
    3    0.500000
    4    0.643559
    dtype: float64,
    'current_value': 0.3333333333333333,
    'reference': None,
    'reference_value': None}},
  {'metric': 'RecallTopKMetric',
   'result': {'k': 3, 'current': 0    0.333333
    1    0.333333
    2    0.666667
    3    1.000000
    dtype: float64, 'current_value': 0.6666666666666666, 'reference': None, 'reference_value': None}},
  {'metric': 'PrecisionTopKMetric',
   'result': {'k': 2, 'current': 0    0.333333
    1    0.166667
    2    0.222222
    3    0.305556
    dtype: float64, 'current_value': 0.16666666666666666, 'reference': None, 'reference_value': None}},
  {'metric': 'FBetaTopKMetric',
   'result': {'k': 2, 'current': 0    0.333333
    1    0.222222
    2    0.333333
    3    0.468085
    dtype: float64, 'current_value': 0.2222222222222222, 'reference': None, 'reference_value': None}},
  {'metric': 'Personalizati